In [1]:
# Operations are defined by manipulating registers with 3 operands.
# The following array of operations is defined by the opcode 
# and lambda functions that takes the registers as first argument.
# for readability purpose, immediate arguments is labeled as `i`
# while ignored argument is labeled as `z`.
#
# Since Julia uses 1-based index, the operands must be added by 1
# to correctly reference the register.
ops = [
    (:addr, (x, a, b, c) -> x[c+1] = x[a+1] + x[b+1]),
    (:addi, (x, a, i, c) -> x[c+1] = x[a+1] + i),
    (:mulr, (x, a, b, c) -> x[c+1] = x[a+1] * x[b+1]),
    (:muli, (x, a, i, c) -> x[c+1] = x[a+1] * i),
    (:banr, (x, a, b, c) -> x[c+1] = x[a+1] & x[b+1]),
    (:bani, (x, a, i, c) -> x[c+1] = x[a+1] & i),
    (:borr, (x, a, b, c) -> x[c+1] = x[a+1] | x[b+1]),
    (:bori, (x, a, i, c) -> x[c+1] = x[a+1] | i),
    (:setr, (x, a, z, c) -> x[c+1] = x[a+1]),
    (:seti, (x, i, z, c) -> x[c+1] = i),
    (:gtir, (x, i, b, c) -> x[c+1] = i > x[b+1] ? 1 : 0),
    (:gtri, (x, a, i, c) -> x[c+1] = x[a+1] > i ? 1 : 0),
    (:gtrr, (x, a, b, c) -> x[c+1] = x[a+1] > x[b+1] ? 1 : 0),
    (:eqir, (x, i, b, c) -> x[c+1] = i == x[b+1] ? 1 : 0),
    (:eqri, (x, a, i, c) -> x[c+1] = x[a+1] == i ? 1 : 0),
    (:eqrr, (x, a, b, c) -> x[c+1] = x[a+1] == x[b+1] ? 1 : 0)
];

In [2]:
# What does the input look like? 
L = readlines("input16_test.txt")

3-element Array{String,1}:
 "Before: [3, 2, 1, 1]"
 "9 2 1 2"             
 "After:  [3, 2, 2, 1]"

In [3]:
# Define parsing functions
parse_before(s) = parse.(Int, match(r"Before: \[(.*), (.*), (.*), (.*)\]", s).captures)
parse_after(s)  = parse.(Int, match(r"After:  \[(.*), (.*), (.*), (.*)\]", s).captures)
parse_instr(s)  = parse.(Int, split(s, " "))

parse_instr (generic function with 1 method)

In [4]:
# Returns number of possible instructions by testing before/instruction/after.
function testme(L1, L2, L3)
    registers, instr, after = parse_before(L1), parse_instr(L2), parse_after(L3)
    count = 0
    for f in ops
        r = copy(registers)
        f[2](r, instr[2], instr[3], instr[4])   # zero-based
        if r == after
            count += 1
        end
    end
    count
end

testme (generic function with 1 method)

In [5]:
# Test sample
testme(L[1], L[2], L[3])

3

In [6]:
# Parse puzzle input.
# The same puzzle input file contains two parts.  The ranges are determined
# by manually bisecting the values.
lines = readlines("input16_1.txt")
puzzle1 = lines[1:3096]
puzzle2 = lines[3099:end]
;

In [7]:
# Part 1 - how many sets of tests have at least 3 possible instructions?
cnt = 0
for i in 1:4:length(puzzle1)
    cnt += (testme(puzzle1[i], puzzle1[i+1], puzzle1[i+2]) >= 3)
end
cnt

517

In [8]:
# Part 2 - Forensic analysis.
# Returns possible instructions.  The `exclude` keyword argument is 
# implemented to remove known instructions so we can quickly matching
# op codes.
function forensic(L1, L2, L3; exclude = Set{Symbol}())
    registers, instr, after = parse_before(L1), parse_instr(L2), parse_after(L3)
    possible = Set{Symbol}()
    for f in ops
        r = copy(registers)
        f[2](r, instr[2], instr[3], instr[4])   # zero-based
        if r == after
            push!(possible, f[1])
        end
    end
    possible = setdiff(possible, exclude)
    if length(possible) == 1
        println("===> hooray, ", instr[1], " = ", possible)
    else
        println(instr[1], " could be ", possible)
    end
    possible
end

forensic (generic function with 1 method)

In [9]:
# Keep changing this code to figure out the op code mappings.
for i in 1:4:length(puzzle1)
    forensic(puzzle1[i], puzzle1[i+1], puzzle1[i+2], 
        exclude = Set([:banr, :bani, :setr, :eqir, :gtir, :eqrr, :gtri, :gtrr, :eqri, :seti, :mulr,
                :addr, :borr, :bori, :addi, :muli]))
end

12 could be Set(Symbol[])
1 could be Set(Symbol[])
2 could be Set(Symbol[])
13 could be Set(Symbol[])
1 could be Set(Symbol[])
9 could be Set(Symbol[])
5 could be Set(Symbol[])
2 could be Set(Symbol[])
10 could be Set(Symbol[])
10 could be Set(Symbol[])
7 could be Set(Symbol[])
12 could be Set(Symbol[])
4 could be Set(Symbol[])
6 could be Set(Symbol[])
3 could be Set(Symbol[])
11 could be Set(Symbol[])
15 could be Set(Symbol[])
6 could be Set(Symbol[])
1 could be Set(Symbol[])
6 could be Set(Symbol[])
1 could be Set(Symbol[])
4 could be Set(Symbol[])
5 could be Set(Symbol[])
14 could be Set(Symbol[])
15 could be Set(Symbol[])
13 could be Set(Symbol[])
15 could be Set(Symbol[])
3 could be Set(Symbol[])
14 could be Set(Symbol[])
6 could be Set(Symbol[])
2 could be Set(Symbol[])
11 could be Set(Symbol[])
0 could be Set(Symbol[])
3 could be Set(Symbol[])
6 could be Set(Symbol[])
1 could be Set(Symbol[])
8 could be Set(Symbol[])
11 could be Set(Symbol[])
7 could be Set(Symbol[])
11 could be

13 could be Set(Symbol[])
15 could be Set(Symbol[])
2 could be Set(Symbol[])
13 could be Set(Symbol[])
1 could be Set(Symbol[])
8 could be Set(Symbol[])
9 could be Set(Symbol[])
7 could be Set(Symbol[])
9 could be Set(Symbol[])
4 could be Set(Symbol[])
13 could be Set(Symbol[])
3 could be Set(Symbol[])
0 could be Set(Symbol[])
2 could be Set(Symbol[])
12 could be Set(Symbol[])
5 could be Set(Symbol[])
4 could be Set(Symbol[])
3 could be Set(Symbol[])
15 could be Set(Symbol[])
1 could be Set(Symbol[])
10 could be Set(Symbol[])
2 could be Set(Symbol[])
0 could be Set(Symbol[])
5 could be Set(Symbol[])
0 could be Set(Symbol[])
3 could be Set(Symbol[])
6 could be Set(Symbol[])
10 could be Set(Symbol[])
6 could be Set(Symbol[])
1 could be Set(Symbol[])
11 could be Set(Symbol[])
7 could be Set(Symbol[])
12 could be Set(Symbol[])
8 could be Set(Symbol[])
9 could be Set(Symbol[])
11 could be Set(Symbol[])
12 could be Set(Symbol[])
13 could be Set(Symbol[])
9 could be Set(Symbol[])
7 could be S

In [10]:
# Final op code mappings
opmap = Dict(5 => :banr, 1 => :bani, 8 => :setr, 4 => :eqir, 12 => :gtir, 10 => :eqrr, 14 => :gtri,
    13 => :gtrr, 0 => :eqri, 2 => :seti, 15 => :mulr, 9 => :addr, 6 => :borr, 3 => :bori, 11 => :addi,
    7 => :muli)

Dict{Int64,Symbol} with 16 entries:
  2  => :seti
  11 => :addi
  0  => :eqri
  9  => :addr
  7  => :muli
  10 => :eqrr
  8  => :setr
  6  => :borr
  4  => :eqir
  3  => :bori
  5  => :banr
  14 => :gtri
  13 => :gtrr
  15 => :mulr
  12 => :gtir
  1  => :bani

In [11]:
# Create a dictionary for easy reference
opsdct = Dict(op[1] => op[2] for op in ops)

Dict{Symbol,Function} with 16 entries:
  :gtrr => ##15#31()
  :mulr => ##5#21()
  :borr => ##9#25()
  :seti => ##12#28()
  :eqri => ##17#33()
  :eqrr => ##18#34()
  :gtri => ##14#30()
  :muli => ##6#22()
  :gtir => ##13#29()
  :setr => ##11#27()
  :addr => ##3#19()
  :banr => ##7#23()
  :eqir => ##16#32()
  :addi => ##4#20()
  :bori => ##10#26()
  :bani => ##8#24()

In [12]:
# Run puzzle program
function runprog(ops, opmap, instructions)
    registers = fill(0, 4)
    for instr in instructions
        opcode = opmap[instr[1]]
        ops[opcode](registers, instr[2], instr[3], instr[4])
    end
    registers
end

runprog(opsdct, opmap, parse_instr.(puzzle2))

4-element Array{Int64,1}:
 667
 667
   3
   2